**Importing the real dataset**                                                           

In [28]:
#importing the deploy data
import numpy as np
import pandas as pd        
deploy = pd.read_excel('Deploy_data.xlsx')
X_deploy = deploy.iloc[:,:4]
X_deploy

,LatDegree,LongDegree,MeasureDepth_m,SurfTemp
0,37.380642,-82.700860,1021.99,14.99
1,37.410835,-82.702323,973.84,14.99
2,37.405493,-82.652496,962.86,15.04
3,37.509499,-82.668528,906.48,14.69
4,37.398042,-82.678669,1130.81,14.99
...,...,...,...,...
95,37.392481,-82.662053,1009.50,15.04
96,37.666696,-82.637182,801.01,14.50
97,37.620100,-82.618600,1019.86,14.50
98,37.656288,-82.711678,789.74,14.47


**Importing the Preprocessors**

In [29]:
#importing scaler and outlier to deploy the given data
import pickle

scl = pickle.load( open( 'Prediction_Scalers.sav', "rb" ) )

out = pickle.load( open( 'Prediction_Outliers.sav', "rb" ) )

**Importing the Best Performing Models**

In [30]:
model_0 = pickle.load( open( f'model_0.sav', "rb" ) )
model_1 = pickle.load( open( f'model_1.sav', "rb" ) )
model_2 = pickle.load( open( f'model_2.sav', "rb" ) )
model_3 = pickle.load( open( f'model_3.sav', "rb" ) )
model_4 = pickle.load( open( f'model_4.sav', "rb" ) )

**Separating the inliers**

In [31]:
#Storing the inlier values in X_i
X_i = X_deploy[out.predict(X_deploy) == 1]
deploy_s = scl.transform(X_i)

C:\Users\Vino\anaconda3\lib\site-packages\sklearn\base.py:457: UserWarning: X has feature names, but IsolationForest was fitted without feature names
  warnings.warn(
C:\Users\Vino\anaconda3\lib\site-packages\sklearn\base.py:457: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


**Prediction of Bottom Hole Temperature using the best performing models**

In [32]:
#prediction of bottom hole temperature with the imported models and deployment inlier data
model_0_p = model_0.predict(deploy_s)
model_1_p = model_1.predict(deploy_s)
model_2_p = model_3.predict(deploy_s)
model_3_p = model_3.predict(deploy_s)
model_4_p = model_4.predict(deploy_s)

**Calculation of Mean and range of the bottom hole temperature**

In [33]:
#calculating the mean and the range of the predictions
all_models_predictions = [model_0_p, model_1_p, model_2_p, model_3_p, model_4_p]
all_models_predictions
mean_predictions = np.mean(all_models_predictions,axis=0)
range_predictions = np.ptp(all_models_predictions,axis=0)

**Exporting the Predicted Values**

In [34]:
#Inserting the mean and range values in the pandas dataframe
predict=pd.DataFrame(scl.inverse_transform(deploy_s),columns=["LatDegree","LongDegree","MeasureDepth_m","SurfTemp"])
predict.insert(4,'Mean_BHT',mean_predictions)
predict.insert(5,'Range_BHT',range_predictions)
#Pandas dataframe to excel
predict.to_excel('Deploy_Results_PX_AJ.xlsx',index=False)